In [5]:
%matplotlib notebook
from mpl_toolkits import mplot3d


import numpy as np
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data 
import time
import math
import random
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [9]:
physicsClient = p.connect(p.GUI)

In [10]:
p.setGravity(0,0,-10) 
p.resetSimulation() 
p.setAdditionalSearchPath(pybullet_data.getDataPath()) 
planeId = p.loadURDF("plane.urdf") 
robotId = p.loadURDF("iiwa7.urdf",flags=9, useFixedBase=1)

robotStartPos = [0,0,0]
robotStartOrientation = p.getQuaternionFromEuler([0,0,0])

p.resetBasePositionAndOrientation(robotId,robotStartPos,robotStartOrientation)

p.setJointMotorControlArray(robotId,range(7),p.VELOCITY_CONTROL,forces=np.zeros(7))

In [11]:
def simulate_system(x, u):
    x_next=[]
    for i in range(7):
        p.resetJointState(robotId,i,x[i],targetVelocity = x[i+7])
    
    p.setJointMotorControlArray(robotId,range(7), controlMode=p.TORQUE_CONTROL,forces=u)
    p.stepSimulation()
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][0])
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][1])
    x_next = np.array(x_next)
    return x_next

In [6]:
N=2000
x = np.zeros([14,N])
u= np.zeros([7, N])
x_new=np.zeros([14,N])

In [12]:
for i in range(N):
    for j in range(7):
        a=random.randint(-180,180)#angule range(-pi,pi)
        b=random.uniform(-10, 10)#velocity range
        c=random.randint(-30,30)#torque range (-200,200)
        x[j,i]=math.radians(a/math.pi)
        x[j+7,i]=b
        u[j,i]=c
            
for i in range(N):
    x_new[:,i]=simulate_system(x[:,i], u[:,i])
        
x=x.T
u = u.T
x_train = np.append(x,u,axis=1)
y_train = x_new.T
x_train = x_train.astype(np.float32)
y_train=y_train.astype(np.float32)
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

print(x_train)

tensor([[  0.2833,   0.5556,  -0.7444,  ..., -19.0000, -30.0000,  -1.0000],
        [ -0.3778,  -0.7667,  -0.6278,  ..., -23.0000,   4.0000,  21.0000],
        [ -0.4222,   0.3889,  -0.6833,  ...,  12.0000, -30.0000,  -4.0000],
        ...,
        [ -0.7778,  -0.6778,  -0.8667,  ...,   4.0000,  -7.0000,  10.0000],
        [  0.3056,   0.7667,   0.2444,  ...,   7.0000,  10.0000,  16.0000],
        [ -0.2222,   0.5556,  -0.5556,  ...,  24.0000,  -9.0000,  20.0000]])


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(21, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 14),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(21, 100)   # hidden layer
        self.predict = torch.nn.Linear(100, 14)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x


class MultiLinearRegression(nn.Module):
    def __init__(self):
        super(MultiLinearRegression, self).__init__()
        self.linear = nn.Linear(21,14)  
        self.hidden
    def forward(self,x):
        out = self.linear(x)
        return out

model = Net()

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

floss=[]
elist=[]
gradlist=[]
epoch = 0
while True:
    output = model(x_train)  
    loss = criterion(output, y_train)  
    loss_value = loss.data.cpu().numpy() 
    optimizer.zero_grad()  
    loss.backward()

    total_norm=0
    for k in model.parameters():
        param_norm = k.grad.data.norm(2)
        total_norm += param_norm.item() ** 2
    total_norm = total_norm ** (1. / 2)
    gradlist.append(total_norm)
    optimizer.step() 
    
    epoch += 1
    if epoch % 1000 == 0: 
        print('Epoch:{}, loss:{:.6f}'.format(epoch, loss_value))
        floss.append(loss_value)
        elist.append(epoch)
    if loss_value <= 1e-1 :
        break

Epoch:1000, loss:1.464097


KeyboardInterrupt: 